# Preparation Script

### Setup

In [29]:
!pip install -r requirements.txt

In [ ]:
!pip install wiki_dump_parser
!pip install pyarrow
!apt install pigz

In [27]:
import wiki_dump_parser
import sys

from os import listdir, remove, system, devnull
from os.path import join
import urllib.request
import pathlib
import subprocess
import pandas as pd


# Disable
def blockPrint():
    sys.stdout = open(devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

dataFolder="./data/"
backupDate="20200401"
language = 'en' # We can also loop through different languages as next step.

### Download Files
First the files will be downloaded from given target.

Then we need to convert xml dumps to csvs for further processing. The files should be residing in ```data/raw/xml```. Don't forget putting xml's under their respective folder, ex: ```data/raw/xml/en/enwiki-20200401-stub-meta-history1.xml```.

In [31]:
def processXmlData(xmlDataFile):
    print(f"Processing {xmlDataFile}")
    blockPrint()
    wiki_dump_parser.xml_to_csv(xmlDataFile)
    enablePrint()
    csvFileName = xmlDataFile.replace(".xml",".csv")
    remove(xmlDataFile)

    # Convert the csv to parquet for faster processing.
    df = pd.read_csv(csvFileName, quotechar ='|')
    df.page_title = df.page_title.astype('string') 
    parquetFileName = csvFileName.replace(".csv", ".parquet")
    df.to_parquet(parquetFileName, engine='pyarrow')
    remove(csvFileName)
    print(f"Completed Processing {xmlDataFile}!")


def processZip(zipPath):
    print(f"Extracting: {zipPath}")
    result = subprocess.run(['unpigz', '-d', zipPath])
    extractedFileName = zipPath.replace(".gz","")
    print(f"Extracted file: {zipPath}")
    processXmlData(extractedFileName)


packs = list(map(lambda x: x+1, range(27)))

for currentPack in packs:
    languageFolder = f"{dataFolder}{language}"
    pathlib.Path(languageFolder).mkdir(parents=True, exist_ok=True)
    
    fileName = f"enwiki-{backupDate}-stub-meta-history{currentPack}.xml.gz"
    backupUrl = f"http://ftp.acc.umu.se/mirror/wikimedia.org/dumps/{language}wiki/{backupDate}/{fileName}"

    filePath = f"{languageFolder}/{fileName}"
    urllib.request.urlretrieve(backupUrl, filePath)
    
    processZip(filePath)
    
    

/opt/anaconda3/envs/wikistory/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


ValueError: StringArray requires a sequence of strings or pandas.NA